## Проект по SQL

### Цель исследования 

Проанализивать базу данных, чтобы в дальнейшем сформулировать ценностное предложение для нового продукта.

### Обзор таблиц 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
# первые строки таблицы books
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# первые строки таблицы authors
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# первые строки таблицы ratings
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
# первые строки таблицы reviews
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
# первые строки таблицы publishers
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Анализ данных

#### Посчитаем, сколько книг вышло после 1 января 2000 года 

In [7]:
query = '''SELECT COUNT(book_id) FROM books WHERE CAST(books.publication_date AS date) > '2000-01-01'; '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


*819 книг вышло после 1 января 2000 года*

#### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
query = '''SELECT books.book_id AS id,
                  books.title AS title,
                  COUNT(DISTINCT reviews.review_id) AS reviews,
                  ROUND(AVG(ratings.rating),1) AS rating 
           FROM books LEFT JOIN ratings ON books.book_id = ratings.book_id LEFT JOIN reviews ON books.book_id = reviews.book_id 
           GROUP BY id, title
           ORDER BY reviews DESC, rating DESC;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,id,title,reviews,rating
0,948,Twilight (Twilight #1),7,3.7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
2,656,The Book Thief,6,4.3
3,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.3
4,734,The Glass Castle,6,4.2
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.0
996,387,Leonardo's Notebooks,0,4.0
997,221,Essential Tales and Poems,0,4.0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7


*Самое большое количество обзоров на книгу "Сумерки" 7, общая оценка 3.7, на втором месте книга "Хоббит, или Туда и обратно" 6 обзоров и оценка 4.4*

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц  для исключения из анализа брошюр; 

In [9]:
query = '''SELECT publishers.publisher AS publisher,
                  COUNT(books.book_id) AS count
           FROM publishers LEFT JOIN books ON publishers.publisher_id = books.publisher_id 
           WHERE books.num_pages > 50
           GROUP BY publisher
           ORDER BY COUNT(book_id) DESC'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


*Издательство Penguin Books которое выпустило наибольшее число книг толще 50 страниц - 42 книги*

#### Определение автора с самой высокой средней оценкой книг,  учитывая только книги с 50 и более оценками; 

In [10]:
query = '''SELECT authors.author AS author,
                  ROUND(AVG(ratings.rating),2) AS rating
           FROM books JOIN ratings ON books.book_id = ratings.book_id LEFT JOIN authors ON books.author_id=authors.author_id
           WHERE books.book_id IN (SELECT books.book_id AS id
                             FROM books JOIN ratings ON books.book_id = ratings.book_id 
                             GROUP BY id
                             HAVING COUNT(ratings.book_id) >= 50)
           GROUP BY author
           ORDER BY rating DESC'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.25
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.79
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
9,Lois Lowry,3.75


*Автор с самой высокой средней оценкой книг(книги с 50 и более оценками) - J.K. Rowling/Mary GrandPré, среднняя оценка 4.29*

#### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок. 

In [11]:
query = '''SELECT AVG(count)
           FROM (SELECT DISTINCT(username) AS us
                 FROM ratings
                 GROUP BY us
                 HAVING COUNT(rating) > 48) AS rating_us
           JOIN
           (SELECT DISTINCT(username) AS us,
                   COUNT(review_id) AS count
            FROM reviews
            GROUP BY us) AS reviews_us ON rating_us.us=reviews_us.us;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


*Среднее количество обзоров от пользователей, который оставили больше 48 оценок составляет 24 обзора.*

### Вывод

Нам передали базу данных крупного сервиса для чтения книг по подписке с целью  проанализировать базу данных , чтобы в дальнейшем сформулировать ценностное предложение для нового продукта.
В базе данных имеют сведения о книгах, издательствах, авторах, а также пользовательские обзоры и оценки книг.

Для формирования предложения, мы сформировали ряд запросов к базу данных, а именно:
Сколько книг вышло после 1 января 2000 года? 
- 819 книг вышло после 1 января 2000 года

Для каждой книги посчитали количество обзоров и среднюю оценку
- Наибольшее количество обзоров и высокую среднюю оценку получили произведения Сумерки с оценкой 3.7; Гарри Поттер и узник Азкабана	с оценкой 4.4, Воровка книг с оценкой 4.3; Гарри Поттер и тайная комната с оценкой 4.3 и Замок из стекла с оценкой 4.2

Определили издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключили из анализа брошюры
- Издательство Penguin Books которое выпустило наибольшее число книг толще 50 страниц - 42 книги;

Определили автора с самой высокой средней оценкой книг — учли только книги с 50 и более оценками
- Автор с самой высокой средней оценкой книг(книги с 50 и более оценками) - J.K. Rowling/Mary GrandPré, среднняя оценка 4.29

Посчитали среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
- Среднее количество обзоров от пользователей, который оставили больше 48 оценок составляет 24 обзора.

На основе выявленных данных можем предложить в качестве первичного предложения для приложения- наиболее популярные книги с высокой средней оценкой(Сумерки с оценкой 3.7; Гарри Поттер и узник Азкабана	с оценкой 4.4, Воровка книг с оценкой 4.3; Гарри Поттер и тайная комната с оценкой 4.3 и Замок из стекла с оценкой 4.2), книги автора с самой высокой оценкой книг- J.K. Rowling/Mary GrandPré.